# News - Online Reports and Images

##  Setup to RDP APIs

In [ ]:
import requests, json, time, getopt, sys

# Application Constants
RDP_version = "/v1"
base_URL = "https://api.refinitiv.com"
category_URL = "/auth/oauth2"
endpoint_URL = "/token"
CLIENT_SECRET = ""
SCOPE = "trapi"
TOKEN_FILE = "token.txt"
TOKEN_ENDPOINT = base_URL + category_URL + RDP_version + endpoint_URL

## Load Credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [ ]:
%run ./credentials.ipynb

### Obtain a Valid Token 

In [ ]:
def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": RDP_LOGIN,
            "password": RDP_PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            APP_KEY,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)

def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];    
    
token = getToken()
token

###  Request Online Reports Hierarchy

In [ ]:
def getOnlineReports():
    news_category_URL = "/data/news"
    reports_endpoint_URL = "/online-reports"


    REQUEST_URL = base_URL + news_category_URL + RDP_version + reports_endpoint_URL

    accessToken = getToken();
    print("Requesting: ",REQUEST_URL)
    
    acceptValue = "application/json"
    dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            return("") 
        accessToken = getToken();     # token refresh on token expired
    else:
        print("Resource access successful")
        return dResp.text
    
txt = getOnlineReports()
jResp = json.loads(txt);
print(json.dumps(jResp, indent=2));


### Select Report Id

In [ ]:
report_id = jResp['data'][0].get('reports')[0].get('reportId')
report_id

In [ ]:
#report_id = "/OLUSTOPNEWS"
def getReportsById(report_ir):
    news_category_URL = "/data/news"
    reports_endpoint_URL = "/online-reports"


    REQUEST_URL = base_URL + news_category_URL + RDP_version + reports_endpoint_URL +  "/" + report_id

    accessToken = getToken();
    print("Requesting: ",REQUEST_URL)
    
    acceptValue = "application/json"
    dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            return("") 
        accessToken = getToken();     # token refresh on token expired
    else:
        print("Resource access successful")
        return dResp.text
    
txt = getReportsById(report_id)
jResp = json.loads(txt);
print(json.dumps(jResp, indent=2));


### Select Image Id (1st)

In [ ]:
myImageId = jResp['data'][0].get('newsItem').get('itemMeta').get('link')[0].get('remoteContent')[0].get('_residref')
myImageId

### Request Online Report Image (1st)

In [ ]:
myImageName = 'uniqueImageName'
def getImage(imageId):
    news_category_URL = "/data/news"
    image_endpoint_URL = "/images/"

    REQUEST_URL = base_URL + news_category_URL + RDP_version + image_endpoint_URL + imageId

    accessToken = getToken();
    print("Requesting: ",REQUEST_URL)
    
    acceptValue = "image/jpeg"
    dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            return("") 
        accessToken = getToken();     # token refresh on token expired
    else:
        print("Resource access successful")
        return dResp.content
    
imgContent = getImage(myImageId)
file = open(myImageName+'.jpg', "wb")
file.write(imgContent)
file.close()

from IPython.display import Image
Image(filename=myImageName+'.jpg') 